In [1]:
from UNet import *
from utils import *

import torch

In [2]:
model = UNet()
model.load_state_dict(torch.load("./hw2_data/face/UNet.pt"))
model = model.cuda()

/tmp/ipykernel_4791/3184638812.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./hw2_data/face/UNet.pt"))


In [3]:
import os
from tqdm import tqdm
from torchvision import transforms as T
from PIL import Image
from torchvision.utils import save_image

noise_dir = "./hw2_data/face/noise"
DDIM_STEP = 50
DDPM_STEP = 1000

STEP = -(DDPM_STEP // DDIM_STEP)


beta = beta_scheduler(1001)
alpha = 1 - beta
alpha_cum = torch.cumprod(alpha, dim=0)

def generate(eta=0.0):
    for path in os.listdir(noise_dir):
        path = os.path.join(noise_dir, path)

        img = torch.load(path)
        
        for t in tqdm(range(DDPM_STEP + STEP + 1, STEP + 1, STEP)):
            with torch.no_grad():
                eps = model(img.cuda(), torch.tensor(t).cuda())

            # alpha_t = torch.full((img.shape[0], 1, 1, 1), (1 - beta_scheduler(1001)[t])).cuda()
            # if t + STEP <= DDPM_STEP:
            #     alpha_t_minus_1 = torch.full((img.shape[0], 1, 1, 1), (1 - beta_scheduler(1001)[t + STEP])).cuda()
            # else:
            #     alpha_t_minus_1 = torch.full((img.shape[0], 1, 1, 1), 1.0).cuda()

            alpha_t = alpha_cum[t]
            alpha_t_minus_1 = alpha_cum[t + STEP if t > -STEP else 0] 
            # alpha_t_minus_1 = alpha_cum[t + STEP] if t > -STEP else torch.tensor(1)

            sigma_t = eta * torch.sqrt((1 - alpha_t_minus_1) / (1 - alpha_t) * (1 - alpha_t / alpha_t_minus_1))

            random_noise = sigma_t * torch.randn_like(img)
            to_xt = (torch.sqrt(1 - alpha_t_minus_1 - sigma_t ** 2) * eps)
            pred_x0 = ((img - torch.sqrt(1 - alpha_t) * eps) / torch.sqrt(alpha_t))

            # pred_x0 = torch.clamp(pred_x0, -1.0, 1.0)
            img = torch.sqrt(alpha_t_minus_1) * pred_x0 + to_xt + random_noise

            # img =  torch.sqrt(alpha_t_minus_1 / alpha_t) * img - torch.sqrt(alpha_t_minus_1 * ( 1 - alpha_t) / alpha_t) * eps + torch.sqrt(1 - alpha_t_minus_1 - sigma_t ** 2) * eps + sigma_t * torch.randn_like(img)
        # img = torch.clamp(img, -1.0, 1.0)
        min_value = img.min()
        img = img - min_value
        max_value = img.max()
        img /= max_value

        save_image(img, os.path.join("p2_outdir", f"{eta}_"+ os.path.basename(path).replace('.pt', ".png")))

## Generate eta 0

In [4]:
generate(0)

/tmp/ipykernel_4791/327528816.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img = torch.load(path)
100%|██████████| 50/50 [00:02<00:00, 20.55it/s]


## calculate MSE

In [5]:
from torch import nn
import torchvision
import os

def load_images2tensor(root_dir):
    img_list = os.listdir(root_dir)
    img_list.sort()
    img = []
    for l in img_list:
        l = os.path.join(root_dir, l)
        i = torchvision.io.read_image(l)
        img.append(i.unsqueeze(0))

    return torch.cat(img)


In [6]:
GT = load_images2tensor("hw2_data/face/GT")
Pd = load_images2tensor("tmp_eta_0/")


In [7]:
((Pd - GT) ** 2).shape

torch.Size([10, 3, 256, 256])

In [8]:
print(torch.mean(((Pd - GT) ** 2).float()), nn.MSELoss()(Pd.float(), GT.float()))

tensor(0.4086) tensor(0.4086)


## Generate Others eta

In [9]:
generate(0.25)
generate(0.5)
generate(0.75)
generate(1)

/tmp/ipykernel_4791/327528816.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img = torch.load(path)
100%|██████████| 50/50 [00:02<00:00, 19.88it/s]


## Make grid

In [10]:
j = 0
while j <= 1:
    print(j)
    j += 0.25

0
0.25
0.5
0.75
1.0


In [11]:
from torchvision.utils import make_grid
from torchvision import transforms as T
from PIL import Image

images = []
j = 0
while j <=1 :
    t = []
    for i in range(4):
        t.append(T.ToTensor()(Image.open(os.path.join("p2_outdir", f"{j if j != 1.0 else '1'}_{i:02}.png"))).unsqueeze(0))
    images.append(torch.concat(t))
    j += 0.25

In [12]:
images = torch.concat(images)

In [13]:
grid = make_grid(images, nrow=4)

In [14]:
save_image(grid, "p2_outdir/out.png")

## interpolation

In [15]:
def inner_product(x, y):
        return torch.sum(x * y, dim=(1, 2, 3))

def get_angle(x, y):
    return torch.arccos(
        inner_product(x, y) / (
            (inner_product(x, x) ** 0.5) * (inner_product(y, y) ** 0.5)
        )
    )[:, None, None, None]

def get_interpolation_weight(start, end, n_points):
    return torch.arange(start, end, n_points)[:, None, None, None].cuda()


def _get_spherically_interpolated_rand_noise(n_points):
    rand_noise1 = torch.load("hw2_data/face/noise/00.pt")
    rand_noise2 = torch.load("hw2_data/face/noise/01.pt")
    ang = get_angle(rand_noise1, rand_noise2)
    weight = get_interpolation_weight(0, 1.1, n_points)

    x_weight = torch.sin((1 - weight) * ang) / torch.sin(ang)
    y_weight = torch.sin(weight * ang) / torch.sin(ang)
    return x_weight * rand_noise1 + y_weight * rand_noise2

def interpolate_in_latent_space(n_points=0.1):
    rand_noise = _get_spherically_interpolated_rand_noise(
        n_points=n_points
    )
    
    return rand_noise

In [16]:
eta = 0
def gen_special_img(noises):
    result = []
    for noise in noises:
        img = noise.unsqueeze(0)
            
        for t in tqdm(range(DDPM_STEP + STEP, STEP, STEP)):
            with torch.no_grad():
                eps = model(img.cuda(), torch.tensor(t).cuda())

            # alpha_t = torch.full((img.shape[0], 1, 1, 1), (1 - beta_scheduler(1001)[t])).cuda()
            # if t + STEP <= DDPM_STEP:
            #     alpha_t_minus_1 = torch.full((img.shape[0], 1, 1, 1), (1 - beta_scheduler(1001)[t + STEP])).cuda()
            # else:
            #     alpha_t_minus_1 = torch.full((img.shape[0], 1, 1, 1), 1.0).cuda()

            alpha_t = alpha_cum[t]
            alpha_t_minus_1 = alpha_cum[t + STEP if t > -STEP else 0] 
            # alpha_t_minus_1 = alpha_cum[t + STEP] if t > -STEP else torch.tensor(1)

            sigma_t = eta * torch.sqrt((1 - alpha_t_minus_1) / (1 - alpha_t) * (1 - alpha_t / alpha_t_minus_1))

            random_noise = sigma_t * torch.randn_like(img)
            to_xt = (torch.sqrt(1 - alpha_t_minus_1 - sigma_t ** 2) * eps)
            pred_x0 = ((img - torch.sqrt(1 - alpha_t) * eps) / torch.sqrt(alpha_t))

            # pred_x0 = torch.clamp(pred_x0, -1.0, 1.0)
            img = torch.sqrt(alpha_t_minus_1) * pred_x0 + to_xt + random_noise

            # img =  torch.sqrt(alpha_t_minus_1 / alpha_t) * img - torch.sqrt(alpha_t_minus_1 * ( 1 - alpha_t) / alpha_t) * eps + torch.sqrt(1 - alpha_t_minus_1 - sigma_t ** 2) * eps + sigma_t * torch.randn_like(img)
        # img = torch.clamp(img, -1.0, 1.0)
        min_value = img.min()
        img = img - min_value
        max_value = img.max()
        img /= max_value

        result.append(img)
        
    return result

In [17]:
result = gen_special_img(interpolate_in_latent_space(0.1))
result = torch.cat(result)

img = make_grid(result, nrow=11)
save_image(img, "p2_outdir/out1.png")

/tmp/ipykernel_4791/3628542764.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rand_noise1 = torch.load("hw2_data/face/noise/00.pt")
/tmp/ipykernel_4791/3628542764.py:1

In [18]:
def linear_noises(step=0.1):
    rand_noise1 = torch.load("hw2_data/face/noise/01.pt")
    rand_noise2 = torch.load("hw2_data/face/noise/00.pt")

    alpha = torch.arange(0, 1.1, step)[:, None, None, None].cuda()
    beta = 1 - alpha

    return rand_noise1 * alpha + rand_noise2 * beta

In [19]:
linear_noises().shape

/tmp/ipykernel_4791/2920519834.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rand_noise1 = torch.load("hw2_data/face/noise/01.pt")
/tmp/ipykernel_4791/2920519834.py:3:

torch.Size([11, 3, 256, 256])

In [20]:
result = gen_special_img(linear_noises(0.1))
result = torch.cat(result)

img = make_grid(result, nrow=11)
save_image(img, "p2_outdir/out2.png")

/tmp/ipykernel_4791/2920519834.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rand_noise1 = torch.load("hw2_data/face/noise/01.pt")
/tmp/ipykernel_4791/2920519834.py:3:

100%|██████████| 50/50 [00:02<00:00, 19.81it/s]
